<a href="https://colab.research.google.com/github/hfathie/qso/blob/master/acc_g_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile test.cu
#include <iostream>
#include <fstream>
using namespace std;

const int N = 70000;
const float G = 1.0f;
const float mSPH = 1.0f / (N/2);


__global__ void acc_g(float *x, float *y, float *z, float *eps, float *accx, float *accy, float *accz, float *mass){
  int i = threadIdx.x + blockIdx.x * blockDim.x;

  if(i < N){

    float dx, dy, dz, rr, inv_r3, epsij, q, q2, q3, q4, q5, q6, fk;
    double accxt = 0.0f, accyt = 0.0f, acczt = 0.0f;
    for(int j = 0; j < N; j++){
      dx = x[j] - x[i];
      dy = y[j] - y[i];
      dz = z[j] - z[i];

      rr = sqrt(dx*dx + dy*dy + dz*dz);
      inv_r3 = 1.0f / (rr*rr*rr);
      epsij = 0.5f * (eps[i] + eps[j]);
      q = rr/epsij;
      q2 = q*q;
      q3 = q2 * q;
      q4 = q3 * q;
      q5 = q4 * q;
      q6 = q5 * q;

      if(q <= 1.0f){
        fk = (1.0f/(epsij*epsij*epsij)) * ((4.0f/3.0f) - (6.0f/5.0f)*q2 + (1.0f/2.0f)*q3);
      }

      if((q > 1.0f) && (q <= 2.0f)){
        fk = inv_r3 * ((-1.0f/15.0f) + (8.0f/3.0f)*q3 - 3.0f*q4 + (6.0f/5.0f)*q5 - (1.0f/6.0f)*q6);
      }

      if(q > 2.0f){
        fk = inv_r3;
      }

      accxt += G * fk * dx * mass[j];
      accyt += G * fk * dy * mass[j];
      acczt += G * fk * dz * mass[j];
    }
    accx[i] = accxt;
    accy[i] = accyt;
    accz[i] = acczt;
  }

}



int main(){

  // Reading the file containing x, y, z, and h.
  ifstream infile("data.csv");
  float xt, yt, zt, ht;
  
  float **data = new float*[N];
  for(int i = 0; i < N; i++){
    data[i] = new float[4];
  }

  for(int i = 0; i < N; i++){
    data[i][0] = 0.0f;
    data[i][1] = 0.0f;
    data[i][2] = 0.0f;
    data[i][3] = 0.0f;
  }

  if(infile.is_open()){
    for(int i = 0; i < N; i++){
      infile >> xt >> yt >> zt >> ht;
      data[i][0] = xt;
      data[i][1] = yt;
      data[i][2] = zt;
      data[i][3] = ht;
    }
  }


  // creating x, y, z arrays in Shared Memorty containing random values between 0 and 1.0
  float *x, *d_x, *y, *d_y, *z, *d_z, *eps, *d_eps, *accx, *accy, *accz, *d_accx, *d_accy, *d_accz, *mass, *d_mass;
  x = new float[N];
  y = new float[N];
  z = new float[N];

  accx = new float[N];
  accy = new float[N];
  accz = new float[N];

  eps = new float[N];
  mass = new float[N];

  cudaMalloc(&d_x, N*sizeof(float));
  cudaMalloc(&d_y, N*sizeof(float));
  cudaMalloc(&d_z, N*sizeof(float));

  cudaMalloc(&d_accx, N*sizeof(float));
  cudaMalloc(&d_accy, N*sizeof(float));
  cudaMalloc(&d_accz, N*sizeof(float));

  cudaMalloc(&d_eps, N*sizeof(float));
  cudaMalloc(&d_mass, N*sizeof(float));

  // Initialize x, y, and z on the Host.
  for(int i = 0; i < N; i++){
    x[i] = data[i][0];
    y[i] = data[i][1];
    z[i] = data[i][2];

    accx[i] = 0.0f;
    accy[i] = 0.0f;
    accz[i] = 0.0f;

    eps[i] = 0.0001f;
    mass[i] = mSPH;
  }

  // Copy from Host to Device.
  cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_accx, accx, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accy, accy, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accz, accz, N*sizeof(float), cudaMemcpyHostToDevice);

  cudaMemcpy(d_eps, eps, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_mass, mass, N*sizeof(float), cudaMemcpyHostToDevice);


  // Launching the kernel on GPU
  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid

  acc_g<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_eps, d_accx, d_accy, d_accz, d_mass);

  // Wait for the GPU to finish before accessing the Host
  cudaDeviceSynchronize();

  // Copy from Device to Host.
  cudaMemcpy(accx, d_accx, N*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(accy, d_accy, N*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(accz, d_accz, N*sizeof(float), cudaMemcpyDeviceToHost);

  // visual inspection
  for(int i = 69990; i < 69999; i++){
    //cout << data[i][0] << ' ' << data[i][1] << ' ' << data[i][2] << endl;
    cout << accx[i] << ' ' << accy[i] << ' ' << accz[i] << endl;
  }

  // Output to a file
  ofstream outfile("acc_g_from_cpp.csv");
  if(outfile.is_open()){
    for(int i = 0; i < N; i++){
      outfile << accx[i] << "," << accy[i] << "," << accz[i] << endl;
    }
  }else cout << "Unable to open file !";

  // Free memory
  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);

  cudaFree(d_accx);
  cudaFree(d_accy);
  cudaFree(d_accz);

  cudaFree(d_eps);
  cudaFree(d_mass);

  delete[] data;
  delete[] x;
  delete[] y;
  delete[] z;

  delete[] accx;
  delete[] accy;
  delete[] accz;

  delete[] eps;
  delete[] mass;

}

Writing test.cu


In [2]:
%%shell
nvcc test.cu -o test

In [3]:
%%shell
./test

-0.631126 0.419961 -0.550773
-0.0120095 0.536762 0.944694
0.842635 0.696079 0.0260362
0.170091 -0.86404 0.486797
0.0522272 -0.117482 -0.723705
0.164239 -0.916709 -0.611625
-1.0732 0.115883 -0.067351
0.738021 0.54477 -0.0451168
-0.719392 0.505126 0.146754


In [4]:
%%shell
nvprof ./test

==174== NVPROF is profiling process 174, command: ./test
-0.631126 0.419961 -0.550773
-0.0120095 0.536762 0.944694
0.842635 0.696079 0.0260362
0.170091 -0.86404 0.486797
0.0522272 -0.117482 -0.723705
0.164239 -0.916709 -0.611625
-1.0732 0.115883 -0.067351
0.738021 0.54477 -0.0451168
-0.719392 0.505126 0.146754
==174== Profiling application: ./test
==174== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.94%  417.27ms         1  417.27ms  417.27ms  417.27ms  acc_g(float*, float*, float*, float*, float*, float*, float*, float*)
                    0.05%  203.55us         8  25.443us  25.247us  26.304us  [CUDA memcpy HtoD]
                    0.02%  67.391us         3  22.463us  22.208us  22.624us  [CUDA memcpy DtoH]
      API calls:   57.96%  417.29ms         1  417.29ms  417.29ms  417.29ms  cudaDeviceSynchronize
                   41.78%  300.81ms         8  37.601ms  2.2090us  300.68ms  cudaMalloc
                

In [16]:
%%writefile test.cu
#include <iostream>
#include <fstream>
using namespace std;

const int N = 70000;
const double G = 1.0f;
const double mSPH = 1.0f / (N/2);


__global__ void acc_g(double *x, double *y, double *z, double *eps, double *accx, double *accy, double *accz, double *mass){
  int i = threadIdx.x + blockIdx.x * blockDim.x;

  if(i < N){

    double dx, dy, dz, rr, inv_r3, epsij, q, q2, q3, q4, q5, q6, fk;
    double accxt = 0.0f, accyt = 0.0f, acczt = 0.0f;
    for(int j = 0; j < N; j++){
      dx = x[j] - x[i];
      dy = y[j] - y[i];
      dz = z[j] - z[i];

      rr = sqrt(dx*dx + dy*dy + dz*dz);
      inv_r3 = 1.0f / (rr*rr*rr);
      epsij = 0.5f * (eps[i] + eps[j]);
      q = rr/epsij;
      q2 = q*q;
      q3 = q2 * q;
      q4 = q3 * q;
      q5 = q4 * q;
      q6 = q5 * q;

      if(q <= 1.0f){
        fk = (1.0f/(epsij*epsij*epsij)) * ((4.0f/3.0f) - (6.0f/5.0f)*q2 + (1.0f/2.0f)*q3);
      }

      if((q > 1.0f) && (q <= 2.0f)){
        fk = inv_r3 * ((-1.0f/15.0f) + (8.0f/3.0f)*q3 - 3.0f*q4 + (6.0f/5.0f)*q5 - (1.0f/6.0f)*q6);
      }

      if(q > 2.0f){
        fk = inv_r3;
      }

      accxt += G * fk * dx * mass[j];
      accyt += G * fk * dy * mass[j];
      acczt += G * fk * dz * mass[j];
    }
    accx[i] = accxt;
    accy[i] = accyt;
    accz[i] = acczt;
  }

}



int main(){

  // Reading the file containing x, y, z, and h.
  ifstream infile("data.csv");
  double xt, yt, zt, ht;
  
  double **data = new double*[N];
  for(int i = 0; i < N; i++){
    data[i] = new double[4];
  }

  for(int i = 0; i < N; i++){
    data[i][0] = 0.0f;
    data[i][1] = 0.0f;
    data[i][2] = 0.0f;
    data[i][3] = 0.0f;
  }

  if(infile.is_open()){
    for(int i = 0; i < N; i++){
      infile >> xt >> yt >> zt >> ht;
      data[i][0] = xt;
      data[i][1] = yt;
      data[i][2] = zt;
      data[i][3] = ht;
    }
  }


  // creating x, y, z arrays in Shared Memorty containing random values between 0 and 1.0
  double *x, *d_x, *y, *d_y, *z, *d_z, *eps, *d_eps, *accx, *accy, *accz, *d_accx, *d_accy, *d_accz, *mass, *d_mass;
  x = new double[N];
  y = new double[N];
  z = new double[N];

  accx = new double[N];
  accy = new double[N];
  accz = new double[N];

  eps = new double[N];
  mass = new double[N];

  cudaMalloc(&d_x, N*sizeof(double));
  cudaMalloc(&d_y, N*sizeof(double));
  cudaMalloc(&d_z, N*sizeof(double));

  cudaMalloc(&d_accx, N*sizeof(double));
  cudaMalloc(&d_accy, N*sizeof(double));
  cudaMalloc(&d_accz, N*sizeof(double));

  cudaMalloc(&d_eps, N*sizeof(double));
  cudaMalloc(&d_mass, N*sizeof(double));

  // Initialize x, y, and z on the Host.
  for(int i = 0; i < N; i++){
    x[i] = data[i][0];
    y[i] = data[i][1];
    z[i] = data[i][2];

    accx[i] = 0.0f;
    accy[i] = 0.0f;
    accz[i] = 0.0f;

    eps[i] = 0.0001f;
    mass[i] = mSPH;
  }

  // Copy from Host to Device.
  cudaMemcpy(d_x, x, N*sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, N*sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, N*sizeof(double), cudaMemcpyHostToDevice);

  cudaMemcpy(d_accx, accx, N*sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accy, accy, N*sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_accz, accz, N*sizeof(double), cudaMemcpyHostToDevice);

  cudaMemcpy(d_eps, eps, N*sizeof(double), cudaMemcpyHostToDevice);
  cudaMemcpy(d_mass, mass, N*sizeof(double), cudaMemcpyHostToDevice);


  // Launching the kernel on GPU
  int blockSize = 256; // number of threads in a block
  int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks in a grid

  acc_g<<<gridSize, blockSize>>>(d_x, d_y, d_z, d_eps, d_accx, d_accy, d_accz, d_mass);

  // Wait for the GPU to finish before accessing the Host
  cudaDeviceSynchronize();

  // Copy from Device to Host.
  cudaMemcpy(accx, d_accx, N*sizeof(double), cudaMemcpyDeviceToHost);
  cudaMemcpy(accy, d_accy, N*sizeof(double), cudaMemcpyDeviceToHost);
  cudaMemcpy(accz, d_accz, N*sizeof(double), cudaMemcpyDeviceToHost);

  // visual inspection
  for(int i = 69990; i < 69999; i++){
    //cout << data[i][0] << ' ' << data[i][1] << ' ' << data[i][2] << endl;
    cout << accx[i] << ' ' << accy[i] << ' ' << accz[i] << endl;
  }

  // Output to a file
  ofstream outfile("acc_g_from_cpp.csv");
  if(outfile.is_open()){
    for(int i = 0; i < N; i++){
      outfile << accx[i] << "," << accy[i] << "," << accz[i] << endl;
    }
  }else cout << "Unable to open file !";

  // Free memory
  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);

  cudaFree(d_accx);
  cudaFree(d_accy);
  cudaFree(d_accz);

  cudaFree(d_eps);
  cudaFree(d_mass);

  delete[] data;
  delete[] x;
  delete[] y;
  delete[] z;

  delete[] accx;
  delete[] accy;
  delete[] accz;

  delete[] eps;
  delete[] mass;

}

Overwriting test.cu


In [17]:
%%shell
nvcc test.cu -o test

In [18]:
%%shell
./test

-0.631124 0.419964 -0.550772
-0.0120095 0.536762 0.944694
0.842635 0.696079 0.0260362
0.170091 -0.86404 0.486797
0.0522273 -0.117482 -0.723705
0.164239 -0.916709 -0.611623
-1.0732 0.115883 -0.0673509
0.738022 0.544769 -0.0451164
-0.719392 0.505126 0.146754


In [19]:
%%shell
nvprof ./test

==370== NVPROF is profiling process 370, command: ./test
-0.631124 0.419964 -0.550772
-0.0120095 0.536762 0.944694
0.842635 0.696079 0.0260362
0.170091 -0.86404 0.486797
0.0522273 -0.117482 -0.723705
0.164239 -0.916709 -0.611623
-1.0732 0.115883 -0.0673509
0.738022 0.544769 -0.0451164
-0.719392 0.505126 0.146754
==370== Profiling application: ./test
==370== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.97%  1.83117s         1  1.83117s  1.83117s  1.83117s  acc_g(double*, double*, double*, double*, double*, double*, double*, double*)
                    0.02%  381.37us         8  47.670us  47.423us  48.063us  [CUDA memcpy HtoD]
                    0.01%  131.68us         3  43.892us  43.550us  44.127us  [CUDA memcpy DtoH]
      API calls:   86.10%  1.83120s         1  1.83120s  1.83120s  1.83120s  cudaDeviceSynchronize
                   13.76%  292.73ms         8  36.591ms  2.6370us  292.51ms  cudaMalloc
      